# 模块加载

In [2]:
!git clone -b dev https://github.com/ARASC/ModelTest.git
!pip install --upgrade pip 
!pip install deepctr

Cloning into 'ModelTest'...
remote: Enumerating objects: 211, done.
remote: Counting objects: 100% (211/211), done.
remote: Compressing objects: 100% (138/138), done.
remote: Total 211 (delta 99), reused 163 (delta 63), pack-reused 0
Receiving objects: 100% (211/211), 40.55 KiB | 233.00 KiB/s, done.
Resolving deltas: 100% (99/99), done.
Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (20.2.3)


In [3]:
%cd ModelTest/

/content/ModelTest


# 获取数据集

In [4]:
from modeltest.dataset.criteo import Criteo
import pandas as pd

In [6]:
# pd.set_option('display.max_columns', None) 

In [5]:
train_size = 1000
test_test = 1000
criteo = Criteo(train_size, test_test)

您可以通过 `criteo.load_data()` 进行数据集的下载  
当检测到默认文件夹中不包含目标数据集时，系统将自动下载数据集  
您可以通过修改 info 文件来自定义储存数据集的默认文件夹


In [6]:
paths = criteo.load_data()
for path in paths:
    print(path)

/root/modeltest_data/criteo/kaggle-display-advertising-challenge-dataset/train.txt
/root/modeltest_data/criteo/kaggle-display-advertising-challenge-dataset/test.txt


您可以通过 `criteo.get_data()` 获取训练和测试数据  
它们将以字典的形式返回 DataFrame 对象


In [7]:
data = criteo.get_data()
print(data['train'].head())

   label   I1   I2    I3  ...       C23       C24       C25       C26
0      0  1.0    1   5.0  ...  3a171ecb  c5c50484  e8b83407  9727dd16
1      0  2.0    0  44.0  ...  3a171ecb  43f13e8b  e8b83407  731c3655
2      0  2.0    0   1.0  ...  3a171ecb  3b183c5c       NaN       NaN
3      0  NaN  893   NaN  ...  3a171ecb  9117a34a       NaN       NaN
4      0  3.0   -1   NaN  ...  32c7478e  b34f3128       NaN       NaN

[5 rows x 40 columns]


# 特征构造

In [8]:
from modeltest.paradigms import CTRFM

In [9]:
embedding_dim = 32
ctrfm = CTRFM()

In [11]:
train_data, test_data = ctrfm.get_data(criteo)
print(train_data.head())

   label        I1        I2        I3        I4  ...  C22  C23  C24  C25  C26
0      0  0.011364  0.000864  0.000591  0.000000  ...    0    1  342   24  183
1      0  0.022727  0.000576  0.005203  0.011494  ...    0    1  111   24  141
2      0  0.022727  0.000576  0.000118  0.160920  ...    3    1   94    0    0
3      0  0.000000  0.257628  0.000000  0.000000  ...    0    1  247    0    0
4      0  0.034091  0.000288  0.000000  0.000000  ...    0    0  323    0    0

[5 rows x 40 columns]


In [12]:
dnn_features, linear_features = ctrfm.get_feature_cols(criteo, embedding_dim)
for feature in dnn_features:
    print(feature)
for feature in linear_features:
         print(feature)

SparseFeat(name='C1', vocabulary_size=53, embedding_dim=32, use_hash=False, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.initializers_v1.RandomNormal object at 0x7f0e5e4959e8>, embedding_name='C1', group_name='default_group', trainable=True)
SparseFeat(name='C2', vocabulary_size=191, embedding_dim=32, use_hash=False, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.initializers_v1.RandomNormal object at 0x7f0e5e495048>, embedding_name='C2', group_name='default_group', trainable=True)
SparseFeat(name='C3', vocabulary_size=674, embedding_dim=32, use_hash=False, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.initializers_v1.RandomNormal object at 0x7f0e5e4950b8>, embedding_name='C3', group_name='default_group', trainable=True)
SparseFeat(name='C4', vocabulary_size=566, embedding_dim=32, use_hash=False, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.initializers_v1.RandomNorm

# 模型训练

# 评价指标